Checking the assigned GPU

In [1]:
!nvidia-smi

Mon Nov 16 22:06:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Cloning the Keras Retinanet Project

In [2]:
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 6205 (delta 1), reused 3 (delta 1), pack-reused 6196
Receiving objects: 100% (6205/6205), 13.48 MiB | 33.17 MiB/s, done.
Resolving deltas: 100% (4196/4196), done.


In [3]:
!pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.4.3)


In [4]:
%cd keras-retinanet/

!pip install .

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp36-cp36m-linux_x86_64.whl size=168077 sha256=69c09304f06d3b902501bfc49beef2f7dfa9c8a0e1e5230c12bab6eeb093b03f
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20487 sha256=e80411166fd2f989c4aa6f73f42d660ec50d743cbf4940aaed4ab1a4e86570cc
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet


Installing the C extensions required be keras retinanet

In [5]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_over

In [6]:
!pip install gdown
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 47kB/s 


In [79]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color



%matplotlib inline
%config inlineBackend.figure_formats='retina'    # It's just that the definition of the displayed plot is a bit better: retina quality. 

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize']=22,10                 # An instance of RcParams for handling default Matplotlib values.

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

Upploaded data (.json) file to google drive. get the shareable link and from that extract the id, this will help you upload the file to colab.

In [8]:
!gdown --id 1eooWl1YZuKSRSPVcXjhyGGvCT9npLRNd --output indian_number_plates.json

Downloading...
From: https://drive.google.com/uc?id=1eooWl1YZuKSRSPVcXjhyGGvCT9npLRNd
To: /content/keras-retinanet/indian_number_plates.json
100% 92.5k/92.5k [00:00<00:00, 28.6MB/s]


Importing .json file as pandas dataframe

In [9]:
plates_df = pd.read_json('indian_number_plates.json', lines=True)

In [10]:
plates_df.head()

,content,annotation,extras
0,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
1,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
2,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
3,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
4,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN


In [11]:
plates_df.shape

(237, 3)

Creating a dictionary to hold all the data from .json file

In [12]:
dataset = dict()
dataset['image_name'] = list()
dataset['x_min'] = list()
dataset['y_min'] = list()
dataset['x_max'] = list()
dataset['y_max'] = list()
dataset['class_name'] = list()

Creating a directory to hold all Images

In [13]:
os.makedirs('number_plates', exist_ok=True)

The .json file provided contains link to all images, below block of code will download all Images, convert it to RGB format, renames the image with index number and then store it in the "number_plates" directory created.

The directory created above is also filled with all the corresponding values.

In [14]:
for index,row in plates_df.iterrows():
  img = urllib.request.urlopen(row['content'])
  img = Image.open(img).convert('RGB')
  img.save(f'./number_plates/licensed_car_{index}.jpeg', 'JPEG')

  dataset['image_name'].append(f'number_plates/licensed_car_{index}.jpeg')

  data = row['annotation']

  dataset['class_name'].append(data[0]['label'][0])

  width = data[0]['imageWidth']
  height = data[0]['imageHeight']

  dataset['x_min'].append(int(round(data[0]['points'][0]['x'] * width)))
  dataset['y_min'].append(int(round(data[0]['points'][0]['y'] * height)))
  dataset['x_max'].append(int(round(data[0]['points'][1]['x'] * width)))
  dataset['y_max'].append(int(round(data[0]['points'][1]['y'] * height)))


In [15]:
df = pd.DataFrame(dataset)
df.head()

,image_name,x_min,y_min,x_max,y_max,class_name
0,number_plates/licensed_car_0.jpeg,582,274,700,321,number_plate
1,number_plates/licensed_car_1.jpeg,80,382,288,449,number_plate
2,number_plates/licensed_car_2.jpeg,41,219,115,260,number_plate
3,number_plates/licensed_car_3.jpeg,217,313,359,359,number_plate
4,number_plates/licensed_car_4.jpeg,209,220,866,554,number_plate


In [16]:
def show_img_object(row):
  img_name = df['image_name'][row]
  img = cv2.imread(img_name)
  img_1 = img.copy()
  img_1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB)
  cv2.rectangle(img_1, (df['x_max'][row], df['y_max'][row]), (df['x_min'][row], df['y_min'][row]), (0,0,255), 2)
  plt.imshow(img_1)
  plt.axis('off')
  plt.show()
  

In [ ]:
show_img_object(0)

In [18]:
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = RANDOM_SEED)

In [19]:
train_df.shape

(189, 6)

In [20]:
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'

In [21]:
train_df.to_csv(ANNOTATIONS_FILE, index=False, header=None)

In [22]:
classes = set(['number_plate'])

with open(CLASSES_FILE, 'w') as csvfile:
  for i,line in enumerate(sorted((classes))):
    csvfile.write('{}, {}\n'.format(line, i))

In [23]:
!head annotations.csv

number_plates/licensed_car_183.jpeg,169,260,358,298,number_plate
number_plates/licensed_car_201.jpeg,251,187,375,234,number_plate
number_plates/licensed_car_230.jpeg,140,149,324,220,number_plate
number_plates/licensed_car_95.jpeg,302,420,671,534,number_plate
number_plates/licensed_car_190.jpeg,115,86,227,117,number_plate
number_plates/licensed_car_84.jpeg,103,209,258,276,number_plate
number_plates/licensed_car_158.jpeg,83,34,369,483,number_plate
number_plates/licensed_car_97.jpeg,123,117,221,170,number_plate
number_plates/licensed_car_147.jpeg,162,227,312,274,number_plate
number_plates/licensed_car_184.jpeg,100,148,259,208,number_plate


#Training

In [24]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloading pretrained model to ' + PRETRAINED_MODEL)

In [26]:
!keras_retinanet/bin/train.py \
 --freeze-backbone \
 --random-transform \
 --weights {PRETRAINED_MODEL} \
 --batch-size 8 \
 --steps 23 \
 --epochs 10 \
 csv annotations.csv classes.csv

2020-11-16 22:07:55.252474: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Creating model, this may take a second...
2020-11-16 22:07:56.692839: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-16 22:07:56.750585: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-16 22:07:56.751221: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-11-16 22:07:56.751264: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-16 22:07:56.945656: I t

#Loading the Trained Model

In [27]:
!ls snapshots/

_pretrained_model.h5  resnet50_csv_04.h5  resnet50_csv_08.h5
resnet50_csv_01.h5    resnet50_csv_05.h5  resnet50_csv_09.h5
resnet50_csv_02.h5    resnet50_csv_06.h5  resnet50_csv_10.h5
resnet50_csv_03.h5    resnet50_csv_07.h5


In [33]:
model_path = os.path.join('snapshots',sorted(os.listdir('snapshots'))[-1])

In [35]:
model_path

'snapshots/resnet50_csv_10.h5'

In [40]:
model = models.load_model(model_path, backbone_name = 'resnet50')
model = models.convert_model(model)

In [41]:
model

In [56]:
labels_to_names = pd.read_csv(CLASSES_FILE, header=None).T.iloc[0].to_dict()

In [57]:
labels_to_names

{0: 'number_plate'}

In [95]:
THRES_SCORE = 0.3

def show_detected_objects(image_row):
  image_path = image_row.image_name
  img = cv2.imread(image_path)

  img_1 = img.copy()
  true_box = [image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max]

  img_1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB)
  
  image = preprocess_image(img)
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  
  boxes /= scale

  draw_box(img_1, true_box, color=(255, 255, 0))

  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
      break

    color = label_color(label)

    b = box.astype(int)
    draw_box(img_1, b, color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(img_1, b, caption)


  plt.axis('off')
  plt.imshow(img_1)
  plt.show()







In [ ]:
show_detected_objects(test_df.iloc[41])